In [1]:
from dotenv import load_dotenv
import dspy
import mlflow

from agentic_system.agents import (
    CFEfficacyAgent,
    ToxicityScreeningAgent,
    CompoundPrioritizationAgent,
)

In [2]:
# NOTE: Start MLflow server with:
# mlflow server --backend-store-uri sqlite:///mydb.sqlite
# Tell MLflow about the server URI.
mlflow.set_tracking_uri("http://127.0.0.1:5000")
# Create a unique name for your experiment.
mlflow.set_experiment("DSPy")
mlflow.autolog()

load_dotenv("../.env")
dspy.configure(
    lm=dspy.LM(
        "gemini/gemini-2.5-flash-lite", temperature=0.5, cache=True, rollout_id=1
    )
)

COMPOUND = "Luminespib"

2025/09/11 11:06:03 INFO mlflow.tracking.fluent: Autologging successfully enabled for dspy.
2025/09/11 11:06:03 INFO mlflow.tracking.fluent: Autologging successfully enabled for litellm.
2025/09/11 11:06:04 INFO mlflow.tracking.fluent: Autologging successfully enabled for openai.


In [3]:
efficacy_agent = CFEfficacyAgent(max_iters=12)
efficacy_results = efficacy_agent(compound_name=COMPOUND)
print(efficacy_results)

Prediction(
    trajectory={'thought_0': "The user wants to estimate the efficacy of Luminespib for reversing cardiac fibrosis. I should start by searching for information about Luminespib's known activities and properties, especially in the context of cardiac fibrosis. I will use `search_chembl_id` to get the ChEMBL ID for Luminespib, which will be useful for subsequent searches.", 'tool_name_0': 'search_chembl_id', 'tool_args_0': {'query': 'Luminespib'}, 'observation_0': "Found 2 compound(s) matching 'Luminespib': \n - CHEMBL3137334 (LUMINESPIB MESYLATE)\n - CHEMBL252164 (LUMINESPIB)", 'thought_1': "I have found two ChEMBL IDs for Luminespib. CHEMBL252164 is the primary ID. I should now use this ID to get more information about the compound's properties and bioactivities. I will start by getting the compound's properties using `get_compound_properties`.", 'tool_name_1': 'get_compound_properties', 'tool_args_1': {'chembl_id': 'CHEMBL252164'}, 'observation_1': "Properties of CHEMBL2521

[Trace(trace_id=94caea1297a941e1ba1674b74166afed), Trace(trace_id=2198f058ed83472381a6d9f60af543b4), Trace(trace_id=702281d178fd4669ad811c4371706e6e), Trace(trace_id=85dac8ea268249ddbd0c2da267f30778), Trace(trace_id=c34715c808474750a0e48cc6c95bd057)]

In [4]:
tox_agent = ToxicityScreeningAgent(max_iters=12)
tox_results = tox_agent(compound_name=COMPOUND)
print(tox_results)

Prediction(
    trajectory={'thought_0': "The user is asking to estimate the toxicity of the compound Luminespib. To do this, I need to find information about its safety and potential toxicity. I will start by searching for the compound's ChEMBL ID. If found, I will use it to retrieve safety information.", 'tool_name_0': 'search_chembl_id', 'tool_args_0': {'query': 'Luminespib'}, 'observation_0': "Found 2 compound(s) matching 'Luminespib': \n - CHEMBL3137334 (LUMINESPIB MESYLATE)\n - CHEMBL252164 (LUMINESPIB)", 'thought_1': 'I have found two ChEMBL IDs for Luminespib. I should investigate both to ensure I have comprehensive information. I will start by getting the safety summary for the first ID, CHEMBL3137334.', 'tool_name_1': 'get_safety_summary', 'tool_args_1': {'cid': 'CHEMBL3137334'}, 'observation_1': 'No safety data available for CID CHEMBL3137334', 'thought_2': 'Since no safety data was found for CHEMBL3137334, I will now try to get the safety summary for the second ChEMBL ID, C

[Trace(trace_id=44e7e6abf34e42a69af4e7b45cc405fa), Trace(trace_id=ee22ac9f4b6e4a9eafddd7dab213ad7e), Trace(trace_id=6d8a23a2297f41dfaf27d2cf31d76c85), Trace(trace_id=79896655f96d4754b65d7d477ea6be47), Trace(trace_id=a197041243a043e783537cca8041e5ac), Trace(trace_id=d96b72f811c54320b501abd2cae5a433), Trace(trace_id=703c73bccce8461194d7d1e8cec05a7b)]

In [5]:
prioritization_agent = CompoundPrioritizationAgent()
prioritization_results = prioritization_agent(compound_name=COMPOUND)
print(prioritization_results)

Prediction(
    reasoning='The compound Luminespib shows high efficacy (0.8) in preclinical studies for cardiac fibrosis, targeting relevant pathways. However, it also exhibits significant toxicity, with only 10% of cells remaining at the tested concentration, indicating potent cytotoxicity. This high toxicity, despite promising efficacy, makes it a moderate-to-low priority for further development without further investigation into dose-response and therapeutic window.',
    priority_score=0.4,
    confidence=0.75
)


[Trace(trace_id=d9505195f6a44aedbf64494e88a9428d), Trace(trace_id=39aaa26b00f54b12b4fb70cceedc54f8)]